Guignard Quentin, Stone Tomas, Hanus Maxime

# Question 1 : Explication de type (1-1) pour la Comparaison x > y

## Formulation Mathématique

### 1. Données et Paramètres
- **x, y** : vecteurs de notes des deux candidats pour chaque cours i
- **w** : vecteur des poids de chaque cours
- **contributions(i)** = w[i] × (x[i] - y[i]) : contribution du cours i à la somme pondérée

### 2. Ensembles
- **pros(x,y)** : ensemble des cours i où x[i] > y[i] (contributions > 0)
- **cons(x,y)** : ensemble des cours i où x[i] < y[i] (contributions < 0)
- **neutral(x,y)** : ensemble des cours i où x[i] = y[i] (contributions = 0)

### 3. Formulation du PL

**Variables de décision:**
- Pour chaque paire (P, C) où P ∈ pros(x,y) et C ∈ cons(x,y) :
  - $x_{P,C} ∈ {0, 1}$ : égal à 1 si la paire (P, C) est sélectionnée comme trade-off

**Contraintes:**
1. Chaque cours $C \in \text{cons}(x,y)$ doit être appairé exactement une fois :
   $$\sum_{P \in \text{pros}(x,y)} x_{P,C} = 1 \quad \forall C \in \text{cons}(x,y)$$

2. Chaque cours $P \in text{pros}(x,y)$ peut être appairé au plus une fois :
   $$\sum_{C \in \text{cons}(x,y)} x_{P,C} \leq 1 \quad \forall P \in \text{pros}(x,y)$$

3. Chaque trade-off sélectionné doit être valide (contribution(P) + contribution(C) > 0) :
   $$(\text{contribution}(P) + \text{contribution}(C)) \cdot x_{P,C} > 0$$
   
   Ou de façon linéaire (si toutes les paires positives) :
   $$\text{contribution}(P) + \text{contribution}(C) > 0 \quad \text{pour chaque paire sélectionnée}$$

**Objectif:**
- Minimiser le nombre de trade-offs : $\min \sum_{P,C} x_{P,C}$
- Ou maximiser pour chercher une explication quelconque : $\max \sum_{P,C} x_{P,C}$

**Certificat de non-existence:**
Si le PL est infaisable (aucune solution), il n'existe pas d'explication (1-1).

In [1]:
import numpy as np
from gurobipy import *

# Données (Xavier vs Yvonne)
courses = ['Anatomie', 'Biologie', 'Chirurgie', 'Diagnostic', 'Epidemiologie', 'Forensic', 'Genetique']
x_notes = np.array([85, 81, 71, 69, 75, 81, 88])
y_notes = np.array([81, 81, 75, 63, 67, 88, 95])
weights = np.array([8, 7, 7, 6, 6, 5, 6])

# Contributions pondérées
contributions = weights * (x_notes - y_notes)
pros = [i for i in range(len(courses)) if contributions[i] > 0]
cons = [i for i in range(len(courses)) if contributions[i] < 0]
neutral = [i for i in range(len(courses)) if contributions[i] == 0]

print('pros:  ', [courses[i] for i in pros])
print('cons:  ', [courses[i] for i in cons])
print('neutral:', [courses[i] for i in neutral])

# Trade-offs (1-1) valides: contribution(P)+contribution(C) > 0
valid_tradeoffs = []
for p in pros:
    for c in cons:
        s = contributions[p] + contributions[c]
        if s > 0:
            valid_tradeoffs.append((p, c, s))

print('trade-offs valides:', [(courses[p], courses[c], s) for p, c, s in valid_tradeoffs])

pros:   ['Anatomie', 'Diagnostic', 'Epidemiologie']
cons:   ['Chirurgie', 'Forensic', 'Genetique']
neutral: ['Biologie']
trade-offs valides: [('Anatomie', 'Chirurgie', np.int64(4)), ('Diagnostic', 'Chirurgie', np.int64(8)), ('Diagnostic', 'Forensic', np.int64(1)), ('Epidemiologie', 'Chirurgie', np.int64(20)), ('Epidemiologie', 'Forensic', np.int64(13)), ('Epidemiologie', 'Genetique', np.int64(6))]


In [2]:
# Résolution PL pour une explication (1-1) minimale
m = Model('explication_1_1')
x_vars = {}
for p, c, _ in valid_tradeoffs:
    x_vars[(p,c)] = m.addVar(vtype=GRB.BINARY, name=f'x_{p}_{c}')
m.update()

# Chaque cons est couvert exactement une fois
for c in cons:
    pairs = [(p, cc) for (p, cc, _) in valid_tradeoffs if cc == c]
    if pairs:
        m.addConstr(quicksum(x_vars[(p, cc)] for (p, cc) in pairs) == 1)

# Chaque pro est utilisé au plus une fois
for p in pros:
    pairs = [(pp, c) for (pp, c, _) in valid_tradeoffs if pp == p]
    if pairs:
        m.addConstr(quicksum(x_vars[(pp, c)] for (pp, c) in pairs) <= 1)

# Objectif: minimiser le nombre de paires
m.setObjective(quicksum(x_vars.values()), GRB.MINIMIZE)
m.params.outputflag = 0
m.optimize()

if m.status == GRB.OPTIMAL:
    print('OPTIMAL, longueur =', int(m.objVal))
    chosen = []
    for (p, c), var in x_vars.items():
        if var.X > 0.5:
            chosen.append((p, c))
            pc = contributions[p]
            cc = contributions[c]
            print(f'({courses[p]}, {courses[c]}) -> {pc} + ({cc}) = {pc+cc}')
else:
    print('Pas de solution (certificat de non-existence).')

Restricted license - for non-production use only - expires 2027-11-29
OPTIMAL, longueur = 3
(Anatomie, Chirurgie) -> 32 + (-28) = 4
(Diagnostic, Forensic) -> 36 + (-35) = 1
(Epidemiologie, Genetique) -> 48 + (-42) = 6
